# Model IO
<img src="https://d.pr/i/Wy5B5B+" width="500"/>

- Language Model
- Prompt
- OutputParser


In [1]:
!pip install langchain langchain-openai langchain-community langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
# langsmith 환경설정
# colab secret 에 모두 등록할 것
from google.colab import userdata
import os
'''
LANGSMITH_TRACING=true
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="<your-api-key>"
LANGSMITH_PROJECT="skn14-langchain"
OPENAI_API_KEY="<your-openai-api-key>"
'''
os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# secret 칸에 5개 적기

## Language Models

https://python.langchain.com/api_reference/reference.html#integrations

LangChain의 Integrations 섹션에서는 다양한 다운스트림 LLM 모델과의 연동을 지원하다.

이 섹션에서는 OpenAI, Hugging Face, GPT-4 등의 다양한 LLM 모델과 LangChain을 연결하는 방법을 다룬다.

In [ ]:
from langchain_openai import ChatOpenAI

#llm=ChatOpenAI(model_name='gpt-4o')
llm.invoke('태국의 수도는 어디인가요?')


### huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(repo_id='microsoft/Phi-3-mini-4k-instruct',
                          task ='text-generation')

chat_model= ChatHuggingFace(
    llm=llm,
    verbose=True
)

chat_model.invoke('Where is the capital of France?')


In [ ]:
from langchain_huggingface import HuggingFacePipeline

pipe = HuggingFacePipeline.from_model_id (
    repo_id='microsoft/Phi-3-mini-4k-instruct',
    task='text-generation'
)
pipe.invoke('What is LLM?')


In [ ]:
# https://python.langchain.com/api_reference/anthropic/llms.html

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatAnthropic(
    model="claude-3-opus-20240229",  # 또는 claude-3-sonnet, claude-3-haiku 등
    temperature=0,
    max_tokens=1024,
    api_key=ANTHROPIC_API_KEY,
)

# 메시지 구성
message = HumanMessage(content="프랑스의 수도는 어디인가요?")

# 응답 생성
response = model.invoke([message])
print(response.content)

### Model Laboratory
- 여러 LLM을 동시에 비교할 수 있는
실험도구

In [ ]:
from langchain.model_laboratory import ModelLaboratory

llms=[
    ChatOpenAI(model_name='gpt-3.5-turbo'),
    ChatOpenAI(model_name='gpt-4.1'),
]

lab=ModelLaboratory.from_llms(llms)
lab.compare('파이썬의 장점이 무엇인가요?')



In [ ]:
## Prompts


https://python.langchain.com/api_reference/core/prompts.html#langchain-core-prompts

`LangChain`의 API 문서에서 제공하는 **Prompts**에 대한 내용은 LangChain 프레임워크의 **핵심 구성 요소 중 하나**로, LLM(Large Language Model)과의 인터페이스를 설정하는 데 중요한 역할을 한다. Prompts는 LLM에 전달될 입력을 정의하고, 구조화하며, 이를 기반으로 원하는 응답을 얻기 위해 사용된다.

**주요 사용처**

1. **자동화된 입력 구성**
   - PromptTemplate을 사용하여 사용자 입력을 자동으로 구성.
   - 동일한 형식의 질문이나 대화를 대량으로 생성 가능.

2. **대화형 응답**
   - ChatPromptTemplate을 통해 대화형 AI의 문맥 유지를 지원.

3. **샘플 기반 학습**
   - Few-shot Prompt는 LLM에 구체적인 예제를 제공해 정확한 응답을 유도.

4. **결과 파싱**
   - Output Parsers를 통해 LLM의 출력을 특정 포맷으로 처리하여 후속 작업을 자동화.


**클래스 계층구조**
```
BasePromptTemplate
├─ PipelinePromptTemplate
├─ StringPromptTemplate
│  ├─ PromptTemplate
│  ├─ FewShotPromptTemplate
│  └─ FewShotPromptWithTemplates
└─ BaseChatPromptTemplate
   ├─ AutoGPTPrompt
   └─ ChatPromptTemplate
      └─ AgentScratchPadChatPromptTemplate

BaseMessagePromptTemplate
├─ MessagesPlaceholder
└─ BaseStringMessagePromptTemplate
   ├─ ChatMessagePromptTemplate
   ├─ HumanMessagePromptTemplate
   ├─ AIMessagePromptTemplate
   └─ SystemMessagePromptTemplate

```

In [ ]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model_name='gpt-4o-mini')
# 환경변수 설정에 os.environ 으로 적혀있어서 api_key 입력 생략

llm.invoke('LLM이 뭔가요?')

In [ ]:
messages = [
    ('system','당신은 친절한 초딩전용 챗봇입니다. 초딩의 눈높이에 맞게 설명해주세요.'),
    ('human', '랭체인이 뭔가요?')

]
llm.invoke(messages)



### PromptTemplate

In [ ]:
from langchain import PromptTemplate

# 어떤 상품에 대한 광고문구를 생성
prompt_template = PromptTemplate(
    template='{product}를 홍보하기 위한 신박한 광고문구를 작성해줘',
    input_variables=["product"]
)
prompt = prompt_template.format(product='초소형 카메라')
prompt = prompt_template.format(product='냉장고')

ai_message = llm.invoke(prompt)
print(ai_message.content)




### ChatPromptTemplate

In [ ]:
from langchain.prompts.chat import (ChatPromptTemplate,
                                    SystemMessagePromptTemplate,
                                    HumanMessagePromptTemplate,
)

system_msg_template = SystemMessagePromptTemplate.from_template("당신은 {domain}분야의 최고의 챗봇입니다.")
human_msg_template = HumanMessagePromptTemplate.from_template("{question}")

chat_template = ChatPromptTemplate.from_messages([
    system_msg_template, human_msg_template])

prompt = chat_template.format_messages(domain='IT', question='LLM이 뭐야?')
prompt

llm.invoke(prompt).content


In [ ]:
prompt = chat_template.format_messages(domain='육아', question='애들이 밥을 잘 안먹는 이유?')
prompt

print(llm.invoke(prompt).content)

### FewShotPromptTemplate

In [ ]:
from langchain.prompts import FewShotPromptTemplate

example = [
    {'q': '2+2=?', 'a': '4'},
    {'q': '3+5=?', 'a': '8'},
]

prompt_template = PromptTemplate(
    template = 'Q: {q}\nA: {a}',
    input_variables=['q','a']
)

fewshot_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt_template,
    prefix='다음 수학문제를 풀어주세요(답변은 정답만 출력하세요):',
    suffix='Q: {question}\nA:', #사용자입력값
    input_variables=['question']
)
prompt = fewshot_template.format(queston='123+345=?')
print(prompt)

In [ ]:
print(llm.invoke(prompt).content)

## Output Parsers

https://python.langchain.com/api_reference/langchain/output_parsers.html#module-langchain.output_parsers

LangChain의 Output Parsers는 LLM이 생성한 텍스트 출력을 특정 형식으로 변환하거나 처리하는 데 사용된다. 이는 모델의 응답을 해석하고, 이를 구조화된 데이터로 바꿔 후속 작업에 활용하기 위해 설계되었다. Output Parsers는 LangChain의 응답 처리 워크플로우에서 중요한 역할을 한다.

예를 들어, LLM 응답이 "Name: John, Age: 30"와 같은 텍스트라면, 이를 {"name": "John", "age": 30}과 같은 Python 딕셔너리로 변환 가능.

**사용 목적**
- 모델의 출력을 특정 애플리케이션에 맞게 처리해야 하는 경우가 많음.
- 응답을 해석하는 일관성과 정확성을 높이기 위해 필요.
- 텍스트 기반 응답을 JSON, 리스트 또는 숫자와 같은 특정 포맷으로 변환하여 후속 작업에 활용.

**종류**
1. **BaseOutputParser**: Output Parsers의 기본 클래스, 커스텀 파서 구현 시 사용.  
2. **CommaSeparatedListOutputParser**: 콤마로 구분된 문자열을 리스트로 변환.  
3. **RegexParser**: 정규식을 사용해 특정 패턴을 추출하고 키-값 형태로 반환.  
4. **StructuredOutputParser**: 출력의 JSON 또는 구조화된 형식을 강제.  
5. **PydanticOutputParser**: Pydantic 모델을 기반으로 출력 검증 및 변환.  
6. **MarkdownOutputParser**: 마크다운 형식의 텍스트에서 데이터를 추출.  

In [ ]:
### CommaSeperatedListOutputParser

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
model_output = "사과, 바나나, 오렌지, 포도"
output_parser = CommaSeparatedListOutputParser()
output = output_parser.parse(model_output)
output


In [ ]:
# 야구팀 5개 질문
# 축구팀 10개 질문

prompt_template = PromptTemplate(
    template="{subject}{n}개의 팀을 보여주세요.\n{format_instruction}",
    input_variables=['subject','n'], # 사용자 프롬프트로 채워질 변수
    partial_variables={
        # template 생성시에 채워짐
        'format_instruction': output_parser.get_format_instructions()
    }
)
prompt = prompt_template.format(subject='대한민국 프로야구',n=5)
prompt = prompt_template.format(subject='프리미어리그',n=5)
prompt

In [ ]:
ai_message = llm.invoke(prompt)
output = ai_message.content
# 출력파서가 가공한 최종출력
output = output_parser.parse(output)
output
# 맨유, 리버풀, 아스널, 첼시, 맨체스터 시티

In [ ]:
chain= prompt_template | llm | output_parser
chain.invoke(input={'subject':'프로농구','n':3})

### JSONOutputParser

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
model_output = '{"title":"GPT-5를 소개합니다.","author":"OpenAI","pages":250}'
#문자열은 모두 쌍따옴표, 숫자는 그대로 쓴다.
json_parser = JsonOutputParser()
print(json_parser.get_format_instructions())

output = json_parser.parse(model_output) # json_str -> python object (list, dict)
print(output)
print(type(output))


#### 연습문제

In [ ]:
# {AI} 관련 책 {3}권을 보여주세요.(json)
# {요리} 관련 책 {5}권을 보여주세요.(json)
# PromptTemplate - LLM - JsonOutputParser


In [ ]:
json_parser = JsonOutputParser()

prompt_template = PromptTemplate(
    template="{subject} 관련 한국어 책 {n}권을 보여주세요.(실제 있는 책만 작성하고 절대 지어내지 마세요.)\n{format_instruction}",
    input_variables=['subject','n'], # 사용자 프롬프트로 채워질 변수
    partial_variables={
        # template 생성시에 채워짐
        'format_instruction': output_parser.get_format_instructions()
    }
)
#prompt = prompt_template.format(subject='요리',n=5)

llm= ChatOpenAI(model_name='gpt-4o-mini')

prompt = prompt_template.format(subject='AI',n=3)

ai_message = llm.invoke(prompt)
output = json_parser.parse(ai_message.content)
print(output)

chain= prompt_template | llm | json_parser
output = chain.invoke(input={'subject':'AI','n':3})
print(output)

# 없는 책 지어내는 경우도 있으므로 주의!



In [ ]:
from langchain_core.output_parsers import JsonOutputParser
model_output = '{"title":"GPT-5를 소개합니다.","author":"OpenAI","pages":250}'
#문자열은 모두 쌍따옴표, 숫자는 그대로 쓴다.
json_parser = JsonOutputParser()
print(json_parser.get_format_instructions())

output = json_parser.parse(model_output) # json_str -> python object (list, dict)
print(output)
print(type(output))
